In [1]:
import surprise
surprise.__version__

'1.1.1'

https://github.com/NicolasHug/Surprise/blob/master/examples/precision_recall_at_k.py

In [2]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
from collections import defaultdict

# from surprise import Dataset
# from surprise import SVD
# from surprise.model_selection import KFold


def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [3]:
import pandas as pd
import numpy as np

df_pred = pd.read_csv('./pred.csv')
df_test = pd.read_csv('./test.csv')

df_pred_new = pd.merge(
    df_pred, 
    df_test.loc[:, ['user_id', 'item_id', 'relevance']], on=['user_id', 'item_id'], how='left'
)

df_pred_new = df_pred_new[df_pred_new.user_id.isin(df_test.user_id)]

df_pred_new.fillna(0, inplace=True)

df_pred_new.loc[df_pred_new.relevance != 0, 'relevance'] = 1

df_pred_new.relevance.value_counts()

0.0    51516
1.0     3144
Name: relevance, dtype: int64

In [4]:
df = pd.merge(df_test, df_pred, how='outer', on=['user_id', 'item_id'])
df = df[df.score.isna() & df.user_id.isin(df_pred.user_id)]

In [5]:
df['score'] = 0

In [7]:
df.head()

,user_id,item_id,rating,timestamp,relevance,score
0,1256,5378,5.0,2007-10-31 12:18:24,5.0,0
1,1256,778,4.5,2007-10-31 12:19:51,4.5,0
2,1256,8376,5.0,2007-10-31 12:30:59,5.0,0
3,1256,2594,5.0,2007-10-31 12:20:49,5.0,0
4,1256,30810,4.5,2007-10-31 12:32:24,4.5,0


In [9]:
pr, re = precision_recall_at_k(
    [
        tuple(x[1].values) + tuple([0])
        for x in pd.concat([df_pred_new, df]).loc[:, ['user_id', 'item_id', 'relevance', 'score']].iterrows()
    ],
    k=20,
    threshold=0.00000000000000000000000000000001
)

In [10]:
np.mean(list(pr.values()))

0.057519209659714604

In [11]:
np.mean(list(re.values()))

0.09632131280310642